In [1]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from test_tool import multiply, load_cv_info
from CV import tailor_CV

# Create the agent
memory = MemorySaver()
model = init_chat_model("openai:gpt-4")
tools = [multiply, load_cv_info, tailor_CV]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Generate a tailored CV.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Generate a tailored CV.
================================== Ai Message ==================================
Tool Calls:
  tailor_CV (call_X2t9fuyYDg6cXKOlfDD4zJ4A)
 Call ID: call_X2t9fuyYDg6cXKOlfDD4zJ4A
  Args:
================================= Tool Message =================================
Name: tailor_CV

Fergal Riordan  
Ireland (Willing to Relocate) | +353 838754381 | fergalriordan333@gmail.com | Portfolio Website | GitHub | LinkedIn  

PROFILE  
Software Engineer with a First-Class Honours MEng in Electronic & Computer Engineering from Trinity College Dublin. Experience spans Agentic AI development for a Partner Relationship Management (PRM) platform, RLHF data annotation for LLMs, and generative AI research in CycleGAN architectures. Proficient in Python and PyTorch, with a passion for building responsible, efficient AI solutions.  

TECHNICAL SKILLS  
Key Competencies: Agentic AI, Generative AI, Prom

In [24]:
from openai import OpenAI
client = OpenAI()

with open("job_description.txt") as f:
    job_listing = f.read()

response = client.responses.create(
   input=f"""
    {job_listing}

    Return the tailored CV as a JSON object with fields:
    Profile, Technical Skills, Relevant Projects
    """,
   prompt={
       "id": "pmpt_68b0c358291c81968a00e9414e386276009a458f13092457",
       "version": "12"
   },
   text={
       "format": {
           "type": "json_schema",
           "name": "cv_schema",
           "schema": {
               "type": "object",
               "properties": {
                   "Profile": {
                       "type": "string",
                       "description": "A tailored CV profile section of approximately 50 words."
                   },
                   "Technical Skills": {
                       "type": "object",
                       "properties": {
                           "Key Competencies": {
                               "type": "array",
                               "items": {"type": "string"},
                               "minItems": 3,
                               "maxItems": 3
                           },
                           "Programming Languages": {
                               "type": "array",
                               "items": {"type": "string"},
                               "minItems": 4,
                               "maxItems": 6
                           },
                           "Frameworks & Libraries": {
                               "type": "array",
                               "items": {"type": "string"},
                               "minItems": 4,
                               "maxItems": 6
                           },
                           "Tools & Platforms": {
                               "type": "array",
                               "items": {"type": "string"},
                               "minItems": 4,
                               "maxItems": 6
                           }
                       },
                       "required": [
                           "Key Competencies",
                           "Programming Languages",
                           "Frameworks & Libraries",
                           "Tools & Platforms"
                       ],
                       "additionalProperties": False
                   },
                   "Relevant Projects": {
                       "type": "array",
                       "items": {
                           "type": "object",
                           "properties": {
                               "Title": {"type": "string"},
                               "Skills": {"type": "string"}
                           },
                           "required": ["Title", "Skills"],
                           "additionalProperties": False
                       },
                       "minItems": 2,
                       "maxItems": 2
                   }
               },
               "required": ["Profile", "Technical Skills", "Relevant Projects"],
               "additionalProperties": False
           },
           "strict": True
       }
   },
)


In [28]:
import json

cv_data = json.loads(response.output_text)
print(json.dumps(cv_data, indent=2))

{
  "Profile": "Software Engineer with expertise in Python and a strong focus on AI and agentic systems. Experience includes developing AI agents for PRM platforms and building scalable automation solutions. Skilled in simplifying complex systems and crafting efficient code. Proven ability to work across diverse tech stacks and contribute to high-impact projects in fast-paced environments.",
  "Technical Skills": {
    "Key Competencies": [
      "Agentic AI",
      "Generative AI",
      "Prompt Engineering"
    ],
    "Programming Languages": [
      "Python",
      "C",
      "C++",
      "SQL"
    ],
    "Frameworks & Libraries": [
      "PyTorch",
      "LangChain",
      "OpenAI API",
      "NumPy"
    ],
    "Tools & Platforms": [
      "Docker",
      "Kubernetes",
      "Azure",
      "Git"
    ]
  },
  "Relevant Projects": [
    {
      "Title": "Master's Thesis: Enhancing CycleGAN for Day-to-Night Image Translation",
      "Skills": "Python, PyTorch, GANs, Transfer Learning,

In [31]:
profile, skills, projects = cv_data["Profile"], cv_data["Technical Skills"], cv_data["Relevant Projects"]

print(f"Profile: {profile}")
print(f"Technical Skills: {skills}")
print(f"Relevant Projects: {projects}")

Profile: Software Engineer with expertise in Python and a strong focus on AI and agentic systems. Experience includes developing AI agents for PRM platforms and building scalable automation solutions. Skilled in simplifying complex systems and crafting efficient code. Proven ability to work across diverse tech stacks and contribute to high-impact projects in fast-paced environments.
Technical Skills: {'Key Competencies': ['Agentic AI', 'Generative AI', 'Prompt Engineering'], 'Programming Languages': ['Python', 'C', 'C++', 'SQL'], 'Frameworks & Libraries': ['PyTorch', 'LangChain', 'OpenAI API', 'NumPy'], 'Tools & Platforms': ['Docker', 'Kubernetes', 'Azure', 'Git']}
Relevant Projects: [{'Title': "Master's Thesis: Enhancing CycleGAN for Day-to-Night Image Translation", 'Skills': 'Python, PyTorch, GANs, Transfer Learning, Deep Learning'}, {'Title': 'Automated Web Development with the DeepSeek API', 'Skills': 'Python, TypeScript, Flask, React, API Integration'}]
